# Import Dependencies

Here We will Connect to SQL Server to Extract Data From Database Call `work` using `pyodbc`

In [ ]:
# pip install pyodbc

In [22]:
import pyodbc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

In [2]:
# List available ODBC drivers
for driver in pyodbc.drivers():
    print(driver)

SQL Server
SQL Server Native Client 11.0
SQL Server Native Client RDA 11.0
Microsoft Access Driver (*.mdb, *.accdb)
Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)
Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)
Microsoft Access Text Driver (*.txt, *.csv)
ODBC Driver 17 for SQL Server
ODBC Driver 18 for SQL Server


In [3]:
# 1. Establish connection to SQL Server (replace placeholders with your credentials)
server = 'DESKTOP-TED1G9O\\SQLEXPRESS01'  # Note the double backslash for escaping
database = 'work'

cnxn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=' + server + ';'
    'DATABASE=' + database + ';'
    'Trusted_Connection=yes;'
)

# 2. Create a cursor object
cursor = cnxn.cursor()

# 3. Execute the SQL query
query = """
Select 
a.ID,
r.Reason,
Case	When Body_mass_index < 18.5 Then 'Underweight'
		When Body_mass_index between 18.5 and 25 Then 'Healthy Weight'
		When Body_mass_index between 25 and 30 Then 'Over Weight'
		Else 'Obese' End as BMI_Category,		-- is a person's weight in kilograms divided by the square of height in meters

Case	When Month_of_absence IN (12, 1, 2) Then 'Winter'
		When Month_of_absence IN (3, 4, 5) Then 'Spring'
		When Month_of_absence IN (6, 7, 8) Then 'Summer'
		When Month_of_absence IN (9, 10, 11) Then 'Fall'
		Else 'Unknow' End as Season_Names,		-- Here we Segment the 4 Seasons
Month_of_absence,
Day_of_the_week,
Transportation_expense,
Son,
Social_drinker,
Social_smoker,
Pet,
Disciplinary_failure,
Age,
Work_load_Average_day,
Hit_target,
Distance_from_Residence_to_Work,
Absenteeism_time_in_hours
From Absenteeism_at_work a
left join compensation c
on a.ID = c.ID
left join Reasons r on a.Reason_for_absence = r.number
"""

cursor.execute(query)

# 4. Fetch the results
results = cursor.fetchall()

# 5. Convert the results to a pandas DataFrame
columns = [column[0] for column in cursor.description]  # Get column names from cursor
df = pd.DataFrame.from_records(results, columns=columns).set_index('ID')

# Optional: Close the cursor and connection
cursor.close()
cnxn.close()


In [4]:
df.head()

,Reason,BMI_Category,Season_Names,Month_of_absence,Day_of_the_week,Transportation_expense,Son,Social_drinker,Social_smoker,Pet,Disciplinary_failure,Age,Work_load_Average_day,Hit_target,Distance_from_Residence_to_Work,Absenteeism_time_in_hours
ID,,,,,,,,,,,,,,,,
1,unjustified absence,Over Weight,Summer,7,3,289,2,True,False,1,False,33,239554,97,36,4
2,Unkown,Obese,Summer,7,3,118,1,True,False,0,True,50,239554,97,13,0
3,medical consultation,Obese,Summer,7,4,179,0,True,False,0,False,38,239554,97,51,2
4,Diseases of the eye and adnexa,Healthy Weight,Summer,7,5,279,2,True,True,0,False,39,239554,97,5,4
5,medical consultation,Over Weight,Summer,7,5,289,2,True,False,1,False,33,239554,97,36,2


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 740 entries, 1 to 740
Data columns (total 16 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Reason                           740 non-null    object
 1   BMI_Category                     740 non-null    object
 2   Season_Names                     740 non-null    object
 3   Month_of_absence                 740 non-null    int64 
 4   Day_of_the_week                  740 non-null    int64 
 5   Transportation_expense           740 non-null    int64 
 6   Son                              740 non-null    int64 
 7   Social_drinker                   740 non-null    bool  
 8   Social_smoker                    740 non-null    bool  
 9   Pet                              740 non-null    int64 
 10  Disciplinary_failure             740 non-null    bool  
 11  Age                              740 non-null    int64 
 12  Work_load_Average_day            740 non-

In [6]:
df.describe()

,Month_of_absence,Day_of_the_week,Transportation_expense,Son,Pet,Age,Work_load_Average_day,Hit_target,Distance_from_Residence_to_Work,Absenteeism_time_in_hours
count,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000
mean,6.324324,3.914865,221.329730,1.018919,0.745946,36.450000,271490.235135,94.587838,29.631081,6.924324
std,3.436287,1.421675,66.952223,1.098489,1.318258,6.478772,39058.116188,3.779313,14.836788,13.330998
min,0.000000,2.000000,118.000000,0.000000,0.000000,27.000000,205917.000000,81.000000,5.000000,0.000000
25%,3.000000,3.000000,179.000000,0.000000,0.000000,31.000000,244387.000000,93.000000,16.000000,2.000000
50%,6.000000,4.000000,225.000000,1.000000,0.000000,37.000000,264249.000000,95.000000,26.000000,3.000000
75%,9.000000,5.000000,260.000000,2.000000,1.000000,40.000000,294217.000000,97.000000,50.000000,8.000000
max,12.000000,6.000000,388.000000,4.000000,8.000000,58.000000,378884.000000,100.000000,52.000000,120.000000


In [7]:
df.shape

(740, 16)

In [8]:
df['Social_smoker'].value_counts()

Social_smoker
False    686
True      54
Name: count, dtype: int64

In [9]:
df['Reason'].value_counts()

Reason
medical consultation                                                                                   149
dental consultation                                                                                    112
physiotherapy                                                                                           69
Diseases of the musculoskeletal system and connective tissue                                            55
Unkown                                                                                                  43
Injury, poisoning and certain other consequences of external causes                                     40
patient follow-up                                                                                       38
unjustified absence                                                                                     33
laboratory examination                                                                                  31
Diseases of the digestive syst

# Data Preprocessing
1. Check Missing Values and Duplicates
2. Feature Encoding: (BMI_Category, Season_Names)
3. Feature Scaling for: (Work_load_Average_day, Hit_target, Transportation_Expenses)
4. Data Visualization:
    - Normal Distribution
    - Outliers Detection
    - Data Analysis
    
4. Feature Selection

5. ML Model Selection

6. Model Evalustion

7. Deploy using Flask

In [10]:
df.isnull().sum()

Reason                             0
BMI_Category                       0
Season_Names                       0
Month_of_absence                   0
Day_of_the_week                    0
Transportation_expense             0
Son                                0
Social_drinker                     0
Social_smoker                      0
Pet                                0
Disciplinary_failure               0
Age                                0
Work_load_Average_day              0
Hit_target                         0
Distance_from_Residence_to_Work    0
Absenteeism_time_in_hours          0
dtype: int64

In [11]:
df.duplicated().sum()

37

In [12]:
duplicated_rows = df[df.duplicated(keep=False)]
duplicated_rows

,Reason,BMI_Category,Season_Names,Month_of_absence,Day_of_the_week,Transportation_expense,Son,Social_drinker,Social_smoker,Pet,Disciplinary_failure,Age,Work_load_Average_day,Hit_target,Distance_from_Residence_to_Work,Absenteeism_time_in_hours
ID,,,,,,,,,,,,,,,,
6,medical consultation,Obese,Summer,7,6,179,0,True,False,0,False,38,239554,97,51,2
17,medical consultation,Obese,Summer,7,6,179,0,True,False,0,False,38,239554,97,51,2
64,medical consultation,Over Weight,Fall,10,3,118,0,False,False,0,False,37,253465,93,10,3
69,medical consultation,Over Weight,Fall,10,3,118,0,False,False,0,False,37,253465,93,10,3
82,medical consultation,Healthy Weight,Fall,11,4,225,1,False,False,2,False,28,306345,93,26,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
674,physiotherapy,Healthy Weight,Spring,4,6,179,0,False,False,0,False,30,246288,91,26,2
681,dental consultation,Healthy Weight,Spring,5,5,291,1,True,False,1,False,40,237656,99,31,2
685,physiotherapy,Healthy Weight,Spring,5,6,179,0,False,False,0,False,30,237656,99,26,2


In [13]:
df.drop_duplicates(inplace=True)

In [16]:
# Create an instance of LabelEncoder
label_encoder = LabelEncoder()

# Encode Season_Names and BMI_Category
df['Season_Names'] = label_encoder.fit_transform(df['Season_Names'])
df['BMI_Category'] = label_encoder.fit_transform(df['BMI_Category'])

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 703 entries, 1 to 740
Data columns (total 16 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Reason                           703 non-null    object
 1   BMI_Category                     703 non-null    int32 
 2   Season_Names                     703 non-null    int32 
 3   Month_of_absence                 703 non-null    int64 
 4   Day_of_the_week                  703 non-null    int64 
 5   Transportation_expense           703 non-null    int64 
 6   Son                              703 non-null    int64 
 7   Social_drinker                   703 non-null    bool  
 8   Social_smoker                    703 non-null    bool  
 9   Pet                              703 non-null    int64 
 10  Disciplinary_failure             703 non-null    bool  
 11  Age                              703 non-null    int64 
 12  Work_load_Average_day            703 non-

In [21]:
df.head()

,BMI_Category,Season_Names,Month_of_absence,Day_of_the_week,Transportation_expense,Son,Social_drinker,Social_smoker,Pet,Disciplinary_failure,Age,Work_load_Average_day,Hit_target,Distance_from_Residence_to_Work,Absenteeism_time_in_hours
ID,,,,,,,,,,,,,,,
1,2,2,7,3,289,2,True,False,1,False,33,239554,97,36,4
2,1,2,7,3,118,1,True,False,0,True,50,239554,97,13,0
3,1,2,7,4,179,0,True,False,0,False,38,239554,97,51,2
4,0,2,7,5,279,2,True,True,0,False,39,239554,97,5,4
5,2,2,7,5,289,2,True,False,1,False,33,239554,97,36,2
